För att köra följande anteckningsböcker, om du inte redan har gjort det, behöver du distribuera en modell som använder `text-embedding-ada-002` som basmodell och ange distributionsnamnet i .env-filen som `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Nästa steg är att vi ska ladda in Embedding Index i en Pandas Dataframe. Embedding Index är sparad i en JSON-fil som heter `embedding_index_3m.json`. Embedding Index innehåller Embeddings för var och en av YouTube-transkriptionerna fram till slutet av oktober 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Nästa steg är att skapa en funktion som heter `get_videos` som söker i Embedding Index efter din fråga. Funktionen returnerar de 5 videor som är mest lika frågan. Så här fungerar funktionen:

1. Först skapas en kopia av Embedding Index.
2. Sedan beräknas Embedding för frågan med hjälp av OpenAI Embedding API.
3. Därefter skapas en ny kolumn i Embedding Index som heter `similarity`. Kolumnen `similarity` innehåller cosinuslikheten mellan Embedding för frågan och Embedding för varje videosegment.
4. Embedding Index filtreras sedan utifrån kolumnen `similarity`. Endast videor med en cosinuslikhet som är lika med eller högre än 0,75 tas med.
5. Slutligen sorteras Embedding Index efter kolumnen `similarity` och de 5 bästa videorna returneras.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Den här funktionen är mycket enkel, den skriver bara ut resultaten av sökfrågan.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Först laddas Embedding Index in i en Pandas Dataframe.
2. Därefter uppmanas användaren att skriva in en sökfråga.
3. Sedan anropas funktionen `get_videos` för att söka i Embedding Index efter sökfrågan.
4. Slutligen anropas funktionen `display_results` för att visa resultaten för användaren.
5. Användaren uppmanas sedan att skriva in en ny sökfråga. Den här processen fortsätter tills användaren skriver `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.sv.png)

Du kommer att bli ombedd att skriva in en sökfråga. Skriv in en fråga och tryck på enter. Applikationen kommer att visa en lista med videor som är relevanta för din fråga. Du får också en länk till den plats i videon där svaret på frågan finns.

Här är några exempel på frågor du kan testa:

- Vad är Azure Machine Learning?
- Hur fungerar konvolutionella neurala nätverk?
- Vad är ett neuralt nätverk?
- Kan jag använda Jupyter Notebooks med Azure Machine Learning?
- Vad är ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Vi strävar efter noggrannhet, men var medveten om att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk ska betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi tar inget ansvar för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
